![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CLINICAL_TEXT_SUMMARIZATION_ES.ipynb)

# Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2  spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [4]:
import json
import os

import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.training import *
from sparknlp.annotator import *

from sparknlp_jsl.base import *
from sparknlp_jsl.annotator import *

from pyspark.ml import Pipeline
import pyspark.sql.functions as F
import pyspark.sql.types as T

import functools 
import numpy as np
import pandas as pd
from scipy import spatial

spark = sparknlp_jsl.start(license_keys['SECRET']) 

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.4.1
Spark NLP_JSL Version : 4.4.1


# summarizer_clinical_jsl_augmented

📌 This model is a modified version of Flan-T5 (LLM) based summarization model that is at first finetuned with natural instructions and then finetuned with clinical notes, encounters, critical care notes, discharge notes, reports, curated  by John Snow Labs. This model is further optimized by augmenting the training methodology, and dataset. It can generate summaries from clinical notes up to 512 tokens given the input text (max 1024 tokens).

⛓️ https://nlp.johnsnowlabs.com/2023/03/30/summarizer_clinical_jsl_augmented_en.html

In [5]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

summarizer = MedicalSummarizer()\
    .pretrained("summarizer_clinical_jsl_augmented", "en", "clinical/models")\
    .setInputCols("document")\
    .setOutputCol("english_summary")\
    .setMaxTextLength(512)\
    .setMaxNewTokens(512)

sentencerDL = SentenceDetectorDLModel.pretrained("sentence_detector_dl", "xx")\
    .setInputCols(["english_summary"])\
    .setOutputCol("sentences")

marian_transformer = MarianTransformer.pretrained("opus_mt_en_es", "xx")\
    .setInputCols(["sentences"])\
    .setOutputCol("spanish_summary")
    
pipeline_summary = Pipeline(stages=[document, 
                                    summarizer, 
                                    sentencerDL, 
                                    marian_transformer])

model_summary= pipeline_summary.fit(spark.createDataFrame([[""]]).toDF("text"))

light_model_summary = LightPipeline(model_summary)

summarizer_clinical_jsl_augmented download started this may take some time.
[OK!]
sentence_detector_dl download started this may take some time.
Approximate size to download 514.9 KB
[OK!]
opus_mt_en_es download started this may take some time.
Approximate size to download 398.8 MB
[OK!]


In [ ]:
text = '''
Fecha: 5 de mayo de 2023

Paciente: Juan Pérez

Edad: 45 años

Motivo de consulta: Dolor abdominal

Antecedentes médicos relevantes: El paciente tiene antecedentes de hipertensión arterial y diabetes tipo 2. Además, ha sido diagnosticado con enfermedad renal crónica en estadio 3.

Examen físico: El paciente se presenta con dolor abdominal difuso, leve sensibilidad a la palpación en la región epigástrica y una presión arterial de 140/90 mmHg.

Resultados de los exámenes: Los resultados de la prueba de hemoglobina glicosilada (HbA1c) son de 7,2%, lo que indica un control adecuado de la diabetes. Sin embargo, se observa una elevación de la creatinina sérica a 1,8 mg/dL, lo que indica un deterioro de la función renal.

Diagnóstico: El paciente presenta dolor abdominal de origen desconocido, pero se debe tener en cuenta la posibilidad de una afección gastrointestinal o renal. Además, el deterioro de la función renal es un hallazgo preocupante que requiere seguimiento y posible ajuste en la terapia farmacológica.

Tratamiento: Se ordenó una ecografía abdominal para descartar una afección gastrointestinal o renal. Además, se realizarán ajustes en la terapia farmacológica para mejorar el control de la presión arterial y el control de la diabetes. Se recomienda una dieta baja en sodio y una ingesta adecuada de líquidos para mejorar la función renal.

Plan de seguimiento: Se programó una cita de seguimiento en dos semanas para revisar los resultados de la ecografía abdominal y evaluar la respuesta al tratamiento. Además, se monitoreará la función renal y se realizarán ajustes en la terapia farmacológica según sea necesario.

Firma:
Dr. Juan García
'''

light_result = light_model_summary.annotate(text)
light_result


{'document': ['\nFecha: 5 de mayo de 2023\n\nPaciente: Juan Pérez\n\nEdad: 45 años\n\nMotivo de consulta: Dolor abdominal\n\nAntecedentes médicos relevantes: El paciente tiene antecedentes de hipertensión arterial y diabetes tipo 2. Además, ha sido diagnosticado con enfermedad renal crónica en estadio 3.\n\nExamen físico: El paciente se presenta con dolor abdominal difuso, leve sensibilidad a la palpación en la región epigástrica y una presión arterial de 140/90 mmHg.\n\nResultados de los exámenes: Los resultados de la prueba de hemoglobina glicosilada (HbA1c) son de 7,2%, lo que indica un control adecuado de la diabetes. Sin embargo, se observa una elevación de la creatinina sérica a 1,8 mg/dL, lo que indica un deterioro de la función renal.\n\nDiagnóstico: El paciente presenta dolor abdominal de origen desconocido, pero se debe tener en cuenta la posibilidad de una afección gastrointestinal o renal. Además, el deterioro de la función renal es un hallazgo preocupante que requiere segu

In [ ]:
spanish_summary = "".join(light_result["spanish_summary"])
spanish_summary

'El paciente tiene 45 años con antecedentes de hipertensión arterial y diabetes tipo 2.Se le ha diagnosticado crónica renal en el estado y tiene antecedentes de hipertensión arterial y de hipertensión arterial.El paciente presenta dolor abdominal, baja presión vascular periférica y alta presión arterial.Los resultados de la prueba de hemoglobina glicosilada son de 7,2%, lo que indica un control adecuado de la diabetes.Sin embargo, una elevación en los niveles de creatinina indica un determinoro de la función renal.El tratamiento incluye una ecografía abdominal para descartar una adección gastrointestinal o renal, ajustes en una terapia farmacológica para mejorar el control de la presión arterial ySe programó una cita de seguimiento para revisar los resultados y evaluar la respuesta al tratamiento.'

In [ ]:
text = '''
Paciente: Ana Pérez
Edad: 10 meses
Género: Femenino
Motivo de consulta: Fiebre y diarrea

Historia clínica:
La paciente Ana Pérez fue llevada a la consulta médica por su madre debido a que presentaba fiebre y diarrea desde hace dos días. La fiebre había estado en torno a los 38ºC y se había mantenido constante, mientras que la diarrea era acuosa y se producía con frecuencia. La paciente también había estado vomitando y no había querido comer.

Antecedentes médicos:
La paciente no tiene antecedentes médicos significativos y estaba recibiendo alimentación materna exclusiva hasta el momento.

Examen físico:
Al examen físico, se observó que la paciente tenía una temperatura de 38.5ºC, y se evidenció deshidratación moderada. El abdomen estaba distendido y sensible a la palpación, y se observó una diarrea acuosa en el pañal.

Diagnóstico:
Basándonos en los síntomas y el examen físico, se sospecha una infección gastrointestinal aguda. Se recomienda realizar un análisis de heces para detectar la presencia de bacterias o parásitos causantes de la infección.

Plan de tratamiento:
Se prescribirá un tratamiento de rehidratación oral con soluciones de sales y líquidos, y se recomendará a la madre que continúe alimentando a la paciente con leche materna. Se prescribirá un medicamento antipirético para controlar la fiebre y se indicará una dieta blanda, evitando alimentos que puedan empeorar la diarrea. Se tomarán muestras de heces para análisis y se prescribirá un tratamiento antibiótico si se detecta la presencia de bacterias patógenas. Se recomendará a la madre seguir de cerca la evolución de la paciente y volver a la consulta si la fiebre persiste o si hay signos de empeoramiento.
'''

light_result = light_model_summary.annotate(text)
light_result

{'document': ['\nPaciente: Ana Pérez\nEdad: 10 meses\nGénero: Femenino\nMotivo de consulta: Fiebre y diarrea\n\nHistoria clínica:\nLa paciente Ana Pérez fue llevada a la consulta médica por su madre debido a que presentaba fiebre y diarrea desde hace dos días. La fiebre había estado en torno a los 38ºC y se había mantenido constante, mientras que la diarrea era acuosa y se producía con frecuencia. La paciente también había estado vomitando y no había querido comer.\n\nAntecedentes médicos:\nLa paciente no tiene antecedentes médicos significativos y estaba recibiendo alimentación materna exclusiva hasta el momento.\n\nExamen físico:\nAl examen físico, se observó que la paciente tenía una temperatura de 38.5ºC, y se evidenció deshidratación moderada. El abdomen estaba distendido y sensible a la palpación, y se observó una diarrea acuosa en el pañal.\n\nDiagnóstico:\nBasándonos en los síntomas y el examen físico, se sospecha una infección gastrointestinal aguda. Se recomienda realizar un 

In [ ]:
spanish_summary = "".join(light_result["spanish_summary"])
spanish_summary

'La paciente, Ana Pérez, tuvo fiebre y diarrea durante 10 días.Llevaba dos días sufriendo fiebre y diarrea, que se mantenía constante mientras la diarrea era acuosa y se producía con frecuencia.El paciente no tenía antecedentes médicos significativos y sólo recibía alimentación materna.El examen físico reveló una moderación deshidratante y se observó diarrea en el área paulal.El plan de tratamiento incluía rehidratación oral con medicamentos, un medicamento para el control bacteriano y una dieta suave, evitando alimentos que pudieran empeoorizar la diarrea.Se tomaron muestras de heces para su análisis, y se prescribió un tratamiento antibiótico si la fiebre persiste o si hay signos de empeooración.Se aconsejó a la madre que siguiera la evolución del paciente y regresara si la fiebre persiste o si hay signos de empeooramiento.'